## Data Augmentation

In [40]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np

# Create an ImageDataGenerator object
datagen = ImageDataGenerator(
    rotation_range=30, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images in a specific class
def augment_images(folder, num_augmented=5):
    images = [img for img in os.listdir(folder) if img.endswith(".jpg") or img.endswith(".png") or img.endswith(".jpeg")]
    for img_name in images:
        img_path = os.path.join(folder, img_name)
        img = Image.open(img_path)
        img = np.array(img)  # Convert to array
        img = img.reshape((1,) + img.shape)  # Reshape for the generator

        # Generate augmented images
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=folder, save_prefix="aug", save_format="jpg"):
            i += 1
            if i >= num_augmented:  # Stop after generating required images
                break

# Apply augmentation on low-sample classes
augment_images("C:\\Users\\121de\\Monument_Final_Year_Latest\\Dataset\\train\\Vishwa_Shanti_Stupa_Nalanda", num_augmented=10)


### Split Dataset

In [42]:
!pip install split-folders

In [43]:
import splitfolders

# Input: Path to your dataset (only train available initially)
input_folder = "C:\\Users\\121de\\Monument_Final_Year_Latest\\Dataset\\train"

# Split into train (70%), validation (15%), and test (15%)
splitfolders.ratio(input_folder, output="monument_dataset", seed=42, ratio=(0.7, 0.15, 0.15))


Copying files: 32522 files [00:51, 632.87 files/s]


### Training and Creating model

In [44]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Path to dataset
train_dir = "C:\\Users\\121de\\Monument_Final_Year_Latest\\monument_dataset\\train"
test_dir = "C:\\Users\\121de\\Monument_Final_Year_Latest\\monument_dataset\\test"

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Compute class weights for imbalanced dataset
class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Load Pretrained Model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Fine-tuning

# Freeze first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Build Model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_data.class_indices), activation='softmax')
])

# Compile Model with Lower Learning Rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)

# Train Model
history = model.fit(train_data, epochs=20, validation_data=test_data, class_weight=class_weights_dict, callbacks=[lr_scheduler])

# Save Model
model.save('monument_recognition_model.h5')


Found 22748 images belonging to 40 classes.
Found 4914 images belonging to 40 classes.


C:\Users\121de\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 1170s 2s/step - accuracy: 0.3224 - loss: 3.9738 - val_accuracy: 0.6249 - val_loss: 1.4412 - learning_rate: 1.0000e-04
Epoch 2/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 860s 1s/step - accuracy: 0.7408 - loss: 1.0010 - val_accuracy: 0.7576 - val_loss: 0.9485 - learning_rate: 1.0000e-04
Epoch 3/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 874s 1s/step - accuracy: 0.8455 - loss: 0.5324 - val_accuracy: 0.8620 - val_loss: 0.4903 - learning_rate: 1.0000e-04
Epoch 4/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 806s 1s/step - accuracy: 0.8943 - loss: 0.3506 - val_accuracy: 0.8938 - val_loss: 0.3413 - learning_rate: 1.0000e-04
Epoch 5/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 792s 1s/step - accuracy: 0.9289 - loss: 0.2413 - val_accuracy: 0.9131 - val_loss: 0.2915 - learning_rate: 1.0000e-04
Epoch 6/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 788s 1s/step - accuracy: 0.9383 - loss: 0.2114 - val_accuracy: 0.9070 - val_loss: 0.3159 - learning_rate: 1.0000e-04
Epoch 7/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 1355s 2s/step - a

In [4]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('monument_recognition_model.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('monument_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\121de\AppData\Local\Temp\tmp11x8tqnv\assets


INFO:tensorflow:Assets written to: C:\Users\121de\AppData\Local\Temp\tmp11x8tqnv\assets


Saved artifact at 'C:\Users\121de\AppData\Local\Temp\tmp11x8tqnv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 40), dtype=tf.float32, name=None)
Captures:
  1654571826448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571818384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571827024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571826832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571826256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571822992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571825680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  165

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory(
    "C:\\Users\\121de\\Monument_Final_Year_Latest\\monument_dataset\\train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

print(train_generator.class_indices)  # This will show the mapping

Found 22748 images belonging to 40 classes.
{'.ipynb_checkpoints': 0, 'Ahilya_Mandir': 1, 'Ashokan_Pillar_Vaishali': 2, 'Bihar_Museum_Patna': 3, 'Biscomaun_Bhawan': 4, 'Black_Buddha_Temple_Nalanda': 5, 'Buddha_Relic_Stupa_Vaishali': 6, 'Buddha_Smriti_Park_Patna': 7, 'Cyclopean_Wall_Rajgir': 8, 'Dashrath_Manjhi_Memorial_Gaya': 9, 'Daud_Khan_Fort_Aurangabad': 10, 'Gandhi_Ghat_Patna': 11, 'Ghora_Katora_lake': 12, 'Golghar': 13, "Griddhakuta_(Vulture's Peak)": 14, 'Hot_Spring_Rajgir': 15, 'Ibrahim_Baya_Maqbara_Nalanda': 16, 'International_Conventional_Hall_Rajgir': 17, 'Jain_Temple_Kundalpur': 18, 'Jal_Mandir_in_Pawapuri': 19, 'Japani_mandir_Rajgir': 20, 'Jarasandha_ka_Akhara_in_Nalanda': 21, 'Kesariya_Stupa': 22, 'Mahabodhi_Temple': 23, 'Maner_Sharief': 24, 'Mundeshwari_Devi_Temple_Kaimur': 25, 'Pandu_Pokhar_Rajgir': 26, 'Ruins_of_Nalanda_University': 27, 'Ruins_of_Vikramshila_Univeristy': 28, 'Sher_Shah_Suri_Tomb': 29, 'Shri_Vishnupad_Temple_Gaya': 30, 'Son_bhandar': 31, 'Sujata_Kutir_Bu

In [49]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a new image to predict
img_path = "C:\\Users\\121de\\Monument_Final_Year_Latest\\monument_dataset\\test\\Son_bhandar\\aug_0_1476.jpg"  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))
model = tf.keras.models.load_model('monument_recognition_model.h5')
# Convert the image to a numpy array and normalize it
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image

# Make a prediction
predictions = model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)

# Map the predicted class index back to the corresponding class label
class_labels = list(train_data.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

print(f"Predicted label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted label: Son_bhandar


In [5]:
import tensorflow as tf
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the quantized model
with open('monument_recognition_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\121de\AppData\Local\Temp\tmppr1ho0_h\assets


INFO:tensorflow:Assets written to: C:\Users\121de\AppData\Local\Temp\tmppr1ho0_h\assets


Saved artifact at 'C:\Users\121de\AppData\Local\Temp\tmppr1ho0_h'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 40), dtype=tf.float32, name=None)
Captures:
  1654571826448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571818384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571827024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571826832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571826256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571822992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571823184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1654571825680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  165